<a href="https://colab.research.google.com/github/docling-project/docling/blob/main/docs/examples/rag_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG with LangChain

| Step | Tech | Execution |
| --- | --- | --- |
| Embedding | Hugging Face / Sentence Transformers | 💻 Local |
| Vector store | Milvus | 💻 Local |
| Gen AI | Hugging Face Inference API | 🌐 Remote |

This example leverages the
[LangChain Docling integration](../../integrations/langchain/), along with a Milvus
vector store, as well as sentence-transformers embeddings.

The presented `DoclingLoader` component enables you to:
- use various document types in your LLM applications with ease and speed, and
- leverage Docling's rich format for advanced, document-native grounding.

`DoclingLoader` supports two different export modes:
- `ExportType.MARKDOWN`: if you want to capture each input document as a separate
  LangChain document, or
- `ExportType.DOC_CHUNKS` (default): if you want to have each input document chunked and
  to then capture each individual chunk as a separate LangChain document downstream.

The example allows exploring both modes via parameter `EXPORT_TYPE`; depending on the
value set, the example pipeline is then set up accordingly.

## Setup

- 👉 For best conversion speed, use GPU acceleration whenever available; e.g. if running on Colab, use GPU-enabled runtime.
- Notebook uses HuggingFace's Inference API; for increased LLM quota, token can be provided via env var `HF_TOKEN`.
- Requirements can be installed as shown below (`--no-warn-conflicts` meant for Colab's pre-populated Python env; feel free to remove for stricter usage):

Les notebooks (Jupyter/Colab) tournent parfois sur un environnement virtuel caché.

La commande shell **!pip** installe parfois les paquets dans le système global de la machine virtuelle, alors que le notebook utilise un environnement Python local.

La commande **%pip** est un alias conçu spécifiquement pour dire "Installe là où ce notebook s'exécute".

In [ ]:
# !pip uninstall -y langchain langchain-classic
# %pip uninstall -y langchain langchain-classic

In [1]:
#%pip install -q --progress-bar off --no-warn-conflicts langchain-docling langchain-core langchain-huggingface langchain_milvus langchain python-dotenv
%pip install -q --progress-bar off langchain-classic langchain-docling langchain-core langchain-huggingface langchain_milvus langchain python-dotenv

  Preparing metadata (setup.py) ... done


In [2]:
!pip list | grep langchain

langchain                                1.2.0
langchain-classic                        1.0.1
langchain-core                           1.2.6
langchain-docling                        2.0.0
langchain-huggingface                    1.2.0
langchain-milvus                         0.3.3
langchain-text-splitters                 1.1.0


In [3]:
import sys
import langchain

print(f"Python executable: {sys.executable}")
print(f"LangChain location: {langchain.__file__}")
# Cela devrait afficher quelque chose comme : .../site-packages/langchain/__init__.py


Python executable: /usr/bin/python3
LangChain location: /usr/local/lib/python3.12/dist-packages/langchain/__init__.py


**Important avec langchain 1.2.0** :
=> Dans cette version de fin 2025), une rupture de compatibilité a été introduite : tout le module langchain.chains a été déplacé dans le paquet langchain-classic pour alléger le cœur du framework.

Le paquet langchain-classic est déjà installé (version 1.0.1),
Il suffit de mettre à jour les imports.

In [4]:
# Au lieu de : from langchain.chains import ...
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain


In [5]:
import os
from pathlib import Path
from tempfile import mkdtemp

from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_docling.loader import ExportType


def _get_env_from_colab_or_os(key):
    try:
        from google.colab import userdata

        try:
            return userdata.get(key)
        except userdata.SecretNotFoundError:
            pass
    except ImportError:
        pass
    return os.getenv(key)


load_dotenv()

# https://github.com/huggingface/transformers/issues/5486:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

HF_TOKEN = _get_env_from_colab_or_os("HF_TOKEN")
FILE_PATH = ["https://arxiv.org/pdf/2408.09869"]  # Docling Technical Report
EMBED_MODEL_ID = "sentence-transformers/all-MiniLM-L6-v2"
#GEN_MODEL_ID = "mistralai/Mixtral-8x7B-Instruct-v0.1"
GEN_MODEL_ID = "mistralai/Mixtral-8x7B-Instruct-v0.1"
EXPORT_TYPE = ExportType.DOC_CHUNKS
QUESTION = "Which are the main AI models in Docling?"
PROMPT = PromptTemplate.from_template(
    "Context information is below.\n---------------------\n{context}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {input}\nAnswer:\n",
)
TOP_K = 3
MILVUS_URI = str(Path(mkdtemp()) / "docling.db")

## Document loading

Now we can instantiate our loader and load documents.

In [6]:
from langchain_docling import DoclingLoader
from docling.chunking import HybridChunker

loader = DoclingLoader(
    file_path=FILE_PATH,
    export_type=EXPORT_TYPE,
    chunker=HybridChunker(tokenizer=EMBED_MODEL_ID),
)

docs = loader.load()

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

[INFO] 2026-01-06 19:51:50,474 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-06 19:51:50,488 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-01-06 19:51:50,495 [RapidOCR] download_file.py:68: Initiating download: https://www.modelscope.cn/models/RapidAI/RapidOCR/resolve/v3.5.0/torch/PP-OCRv4/det/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-06 19:51:51,630 [RapidOCR] download_file.py:82: Download size: 13.83MB
[INFO] 2026-01-06 19:51:52,387 [RapidOCR] download_file.py:95: Successfully saved to: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-06 19:51:52,392 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-06 19:51:53,089 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-06 19:51:53,091 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-01-06 19:51:53,094 [RapidOCR] download_file.py:68: Initiat

> Note: a message saying `"Token indices sequence length is longer than the specified
maximum sequence length..."` can be ignored in this case — details
[here](https://github.com/docling-project/docling-core/issues/119#issuecomment-2577418826).

Determining the splits:

In [7]:
if EXPORT_TYPE == ExportType.DOC_CHUNKS:
    splits = docs
elif EXPORT_TYPE == ExportType.MARKDOWN:
    from langchain_text_splitters import MarkdownHeaderTextSplitter

    splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=[
            ("#", "Header_1"),
            ("##", "Header_2"),
            ("###", "Header_3"),
        ],
    )
    splits = [split for doc in docs for split in splitter.split_text(doc.page_content)]
else:
    raise ValueError(f"Unexpected export type: {EXPORT_TYPE}")

Inspecting some sample splits:

In [8]:
for d in splits[:3]:
    print(f"- {d.page_content=}")
print("...")

- d.page_content='Version 1.0\nChristoph Auer Maksym Lysak Ahmed Nassar Michele Dolfi Nikolaos Livathinos Panos Vagenas Cesar Berrospi Ramis Matteo Omenetti Fabian Lindlbauer Kasper Dinkla Lokesh Mishra Yusik Kim Shubham Gupta Rafael Teixeira de Lima Valery Weber Lucas Morin Ingmar Meijer Viktor Kuropiatnyk Peter W. J. Staar\nAI4K Group, IBM Research R¨ uschlikon, Switzerland'
- d.page_content='Abstract\nThis technical report introduces Docling , an easy to use, self-contained, MITlicensed open-source package for PDF document conversion. It is powered by state-of-the-art specialized AI models for layout analysis (DocLayNet) and table structure recognition (TableFormer), and runs efficiently on commodity hardware in a small resource budget. The code interface allows for easy extensibility and addition of new features and models.'
- d.page_content='1 Introduction\nConverting PDF documents back into a machine-processable format has been a major challenge for decades due to their huge vari

## Ingestion

In [9]:
%pip install pymilvus[milvus_lite]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 16.2 MB/s eta 0:00:00


In [10]:
import json
from pathlib import Path
from tempfile import mkdtemp

from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_milvus import Milvus

embedding = HuggingFaceEmbeddings(model_name=EMBED_MODEL_ID)

milvus_uri = str(Path(mkdtemp()) / "docling.db")  # or set as needed
vectorstore = Milvus.from_documents(
    documents=splits,
    embedding=embedding,
    collection_name="docling_demo",
    connection_args={"uri": milvus_uri},
    index_params={"index_type": "FLAT"},
    drop_old=True,
)

## RAG

In [ ]:
#!pip list | grep langchain

In [16]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_huggingface import HuggingFaceEndpoint

retriever = vectorstore.as_retriever(search_kwargs={"k": TOP_K})
llm = HuggingFaceEndpoint(
    repo_id=GEN_MODEL_ID,
    huggingfacehub_api_token=HF_TOKEN,
    pipeline_kwargs={
        "max_new_tokens": 500,
        "temperature": 0.7,
        "top_p": 0.95,
        "do_sample": True,
        "return_full_text": False  # ⚠️ CRITIQUE
    }
)

def clip_text(text, threshold=100):
    return f"{text[:threshold]}..." if len(text) > threshold else text

                    pipeline_kwargs was transferred to model_kwargs.
                    Please make sure that pipeline_kwargs is what you intended.


In [17]:
question_answer_chain = create_stuff_documents_chain(llm, PROMPT)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)
resp_dict = rag_chain.invoke({"input": QUESTION})

clipped_answer = clip_text(resp_dict["answer"], threshold=200)
print(f"Question:\n{resp_dict['input']}\n\nAnswer:\n{clipped_answer}")
for i, doc in enumerate(resp_dict["context"]):
    print()
    print(f"Source {i + 1}:")
    print(f"  text: {json.dumps(clip_text(doc.page_content, threshold=350))}")
    for key in doc.metadata:
        if key != "pk":
            val = doc.metadata.get(key)
            clipped_val = clip_text(val) if isinstance(val, str) else val
            print(f"  {key}: {clipped_val}")

KeyboardInterrupt: 

**Autre exemple** :

In [ ]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_huggingface import HuggingFacePipeline
from langchain_classic.prompts import ChatPromptTemplate

# Configuration du LLM local
llm = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    task="text-generation",  # ✅ Correct pour modèle local
    pipeline_kwargs={
        "max_new_tokens": 500,
        "temperature": 0.7,
        "top_p": 0.95,
        "do_sample": True,
        "return_full_text": False  # ⚠️ CRITIQUE
    }
)

# Saisie manuelle
# context = input("Entrez le contexte: ")
# question = input("Entrez la question: ")

PROMPT = ChatPromptTemplate.from_template("""
Context: {context}
Question: {input}
Answer:""")

question_answer_chain = create_stuff_documents_chain(llm, PROMPT)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)
resp_dict = rag_chain.invoke({"input": QUESTION})

# Affichage des résultats
clipped_answer = clip_text(resp_dict["answer"], threshold=200)
print(f"Question:\n{resp_dict['input']}\n\nAnswer:\n{clipped_answer}")

for i, doc in enumerate(resp_dict["context"]):
    print(f"\nSource {i + 1}:")
    print(f"  text: {json.dumps(clip_text(doc.page_content, threshold=350))}")
    for key in doc.metadata:
        if key != "pk":
            val = doc.metadata.get(key)
            clipped_val = clip_text(val) if isinstance(val, str) else val
            print(f"  {key}: {clipped_val}")
